# IMPORTING LIBRARIES

In [1]:
import boto3
import os
import io
import numpy as np
import logging
import tempfile
import pandas as pd
import random
import sagemaker
import datetime

from botocore.exceptions import NoCredentialsError
from sagemaker.pytorch import PyTorch
from sagemaker.model import ModelPackage
from sagemaker.estimator import Estimator
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.functions import Join
from sagemaker.workflow.lambda_step import LambdaStep
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import ModelMetrics
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.processing import (
    ScriptProcessor, 
    FrameworkProcessor, 
    ProcessingInput, 
    ProcessingOutput
)
from sagemaker.workflow.steps import (
    TrainingStep, 
    ProcessingStep
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.workflow.parameters import ( 
    ParameterInteger, 
    ParameterString, 
    ParameterFloat
)

SAGEMAKER_SESSION = sagemaker.Session()
PIPELINE_SESSION = PipelineSession()
ROLE = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# SETTING UP INPUTS AND IMAGES

In [2]:
#Pre-processing inputs
s3_bucket_param = ParameterString(name="S3Bucket", default_value="yolo-wildfire-smoke-detection")
s3_folder_param = ParameterString(name="S3Folder", default_value="wildfire_smoke")
train_ratio_param = ParameterFloat(name="TrainTestRatio", default_value=0.8)

#Training inputs
model_param = ParameterString(name='Model', default_value='yolov9m.pt')
epochs_param = ParameterInteger(name='Epochs', default_value=10)
batch_size_param = ParameterInteger(name='BatchSize', default_value=16)
patience_param = ParameterInteger(name='Patience', default_value=100)
optimizer_param = ParameterString(name='Optimizer', default_value='auto')
ilr_param = ParameterFloat(name='InitialLearningRate', default_value=0.01)
flr_param = ParameterFloat(name='FinalLearningRate', default_value=0.01)

#Metrics treholds & Model packing inputs
mAP_threshold = ParameterFloat(name='mAPThreshold', default_value=0.75)
mAP50_threshold = ParameterFloat(name='mAP50Threshold', default_value=0.90)
mAP75_threshold = ParameterFloat(name='mAP75Threshold', default_value=0.80)
precisionthreshold = ParameterFloat(name='PrecisionThreshold', default_value=0.90)
recallthreshold = ParameterFloat(name='RecallThreshold', default_value=0.80)
model_package_name = ParameterString(name="ModelPackageName", default_value='yolo-smoke-detection')


#Images
PROCESSING_IMAGE = sagemaker.image_uris.retrieve(framework='pytorch', 
                                          region=SAGEMAKER_SESSION.boto_region_name, 
                                          version='2.2.0', 
                                          py_version='py310',
                                          instance_type='ml.m5.2xlarge',
                                          image_scope='training')

TRAINING_IMAGE = sagemaker.image_uris.retrieve(framework='pytorch', 
                                          region=SAGEMAKER_SESSION.boto_region_name, 
                                          version='2.2.0', 
                                          py_version='py310', 
                                          instance_type='ml.c5.2xlarge',
                                          image_scope='training')

# PRE-PROCESSING STEP

In [3]:
script_processor = ScriptProcessor(
    image_uri=PROCESSING_IMAGE,
    command=['python3'],
    role=ROLE,
    instance_count=1,
    instance_type='ml.t3.xlarge',
    sagemaker_session=SAGEMAKER_SESSION
)

processing_step = ProcessingStep(
    name='DataPreProcessing',
    processor=script_processor,
    inputs=[
        ProcessingInput(
            source=Join(on='/', values=["s3:/", s3_bucket_param, s3_folder_param]),
            destination='/opt/ml/processing/input'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='train_data', 
            source='/opt/ml/processing/train', 
            destination=Join(on='/', values=["s3:/", s3_bucket_param, s3_folder_param, "train_subset"])
        ),
        ProcessingOutput(
            output_name='val_data', 
            source='/opt/ml/processing/val', 
            destination=Join(on='/', values=["s3:/", s3_bucket_param, s3_folder_param, "val_subset"])
        ),
        ProcessingOutput(
            output_name='test_data', 
            source='/opt/ml/processing/test', 
            destination=Join(on='/', values=["s3:/", s3_bucket_param, s3_folder_param, "test_subset"])
        )
    ],
    code='smoke_preprocess.py',
    job_arguments=[
        '--s3-bucket', s3_bucket_param.to_string(),
        '--s3-folder', s3_folder_param.to_string(),
        '--train-ratio', train_ratio_param.to_string()
    ]
)


# TRAINING STEP

In [4]:
S3_MODEL_REGISTRY_PATH = "s3://smoke-detection-model-registry/packaged-models/" #Adjust to your preference

training_estimator = Estimator(image_uri=TRAINING_IMAGE,
                      role=ROLE,
                      entry_point='smoke_train.py',
                      instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      hyperparameters={
                          'model': model_param,
                          'epochs': epochs_param,
                          'batch': batch_size_param,
                          'patience': patience_param,
                          'optimizer': optimizer_param,
                          'initial_learning_rate': ilr_param,
                          'final_learning_rate': flr_param,
                      },
                      output_path=S3_MODEL_REGISTRY_PATH,
                      sagemaker_session=SAGEMAKER_SESSION,
                      source_dir=".")

training_step = TrainingStep(
    name="TrainingYOLOModel",
    estimator=training_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs['train_data'].S3Output.S3Uri,
            content_type="application/x-recordio"
        ),
        "val": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs['val_data'].S3Output.S3Uri,
            content_type="application/x-recordio"
        )
    }
)

# EVALUATION STEP

In [5]:
FOLDER_NAME = f"val-results-{datetime.datetime.now()}" #Adjust to your preference

script_eval = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version='2.2.0',
    py_version='py310',
    instance_type='ml.t3.xlarge',
    instance_count=1,
    base_job_name='yolo-eval-test',
    role=ROLE,
    sagemaker_session=PIPELINE_SESSION,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/model',
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs['test_data'].S3Output.S3Uri,
            destination='/opt/ml/processing/input'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/evaluation',
            destination=f's3://<bucket-name-to-store-eval-results>/{FOLDER_NAME}' #Adjust <bucket-name-to-store-eval-results> to your preference
        )
    ],
    code='smoke_evaluate.py'
)

evaluation_step = ProcessingStep(
    name='EvaluationYOLOModel',
    step_args=eval_args
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [6]:
LAMBDA_FUNCTION_ARN = "arn:aws:lambda:us-east-1:810786622544:function:smoke-evaluate-metrics"
lambda_function = Lambda(function_arn=LAMBDA_FUNCTION_ARN)

result = LambdaOutput(output_name="result", output_type=LambdaOutputTypeEnum.Boolean)
mAP = LambdaOutput(output_name="mAP", output_type=LambdaOutputTypeEnum.String)
mAP50 = LambdaOutput(output_name="mAP50", output_type=LambdaOutputTypeEnum.String)
mAP75 = LambdaOutput(output_name="mAP75", output_type=LambdaOutputTypeEnum.String)
precision = LambdaOutput(output_name="precision", output_type=LambdaOutputTypeEnum.String)
recall = LambdaOutput(output_name="recall", output_type=LambdaOutputTypeEnum.String)

lambda_step = LambdaStep(
    name="AnalyzeYOLOMetrics",
    lambda_func=lambda_function,
    inputs={
        "s3_uri": evaluation_step.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
        "mAPThreshold": mAP_threshold,
        "mAP50Threshold": mAP50_threshold,
        "mAP75Threshold": mAP75_threshold,
        "precisionThreshold": precisionthreshold,
        "recallThreshold": recallthreshold
    },
    outputs=[result, mAP, mAP50, mAP75, precision, recall]
)

# STEP FOR CONDITIONAL MODEL REGISTRY PUSH

In [7]:
METRICS_PATH = f's3://smoke-detection-eval-metrics/{FOLDER_NAME}/metrics.json' #Adjust to your preference

register_estimator = Estimator(image_uri=PROCESSING_IMAGE,
                      role=ROLE,
                      instance_count=1,
                      instance_type='ml.t3.medium',
                      sagemaker_session=SAGEMAKER_SESSION
                     )

register_model_step = RegisterModel(
    name="YOLOTrainedModel",
    estimator=register_estimator,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_name,
    approval_status="PendingManualApproval",
    description=f"Model metrics available at {METRICS_PATH}"
)

condition_check = ConditionEquals(
    left=lambda_step.properties.Outputs['result'], 
    right=True
)

registry_condition_step = ConditionStep(
    name="YOLOModelRegistration",
    conditions=[condition_check],
    if_steps=[register_model_step],
    else_steps=[]
)

# SETUP FINAL PIPELINE WITH ALL STEPS AND INPUTS

In [8]:
PIPELINE_NAME = "Smoke-Detection-Yolo-Pipeline"

pipeline = Pipeline(
    name=PIPELINE_NAME,
    parameters=[
        s3_bucket_param,
        s3_folder_param,
        train_ratio_param,
        model_param,
        epochs_param,
        batch_size_param,
        patience_param,
        optimizer_param,
        ilr_param,
        flr_param,
        mAP_threshold,
        mAP50_threshold,
        mAP75_threshold,
        precisionthreshold,
        recallthreshold,
        model_package_name
    ],
    steps=[processing_step, training_step, evaluation_step, lambda_step, registry_condition_step]
)

pipeline.upsert(role_arn=ROLE)

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-810786622544/Smoke-Detection-Yolo-Pipeline/code/a079018d2224bfa13504eab4a1bd4a07/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-810786622544/Smoke-Detection-Yolo-Pipeline/code/0ff95e6702f0700d1aa530170082a883/runproc.sh
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/lambda_step.py:165: UserWarning: Lambda function won't be updated because zipped_code_dir                 or script is not provided.
  warnings.warn(
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-810786622544/Smoke-Detection-Yolo-Pipeline/code/a079018d2224bfa13504eab4a1bd4a07/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-810786622544/Smoke-Detection-Yolo-Pipeline/code/0ff95e6702f0700d1aa530170082a883/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:810786622544:pipeline/Smoke-Detection-Yolo-Pipeline',
 'ResponseMetadata': {'RequestId': 'c1baac81-a4f3-4429-8a72-5fbbf0ad10ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c1baac81-a4f3-4429-8a72-5fbbf0ad10ae',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Tue, 18 Jun 2024 15:06:23 GMT'},
  'RetryAttempts': 0}}